In [1]:
import pandas as pd
from census import Census

In [35]:
# Get Variable Code
def get_acs_code(var_select,filepath = '../data/acs_variable_code.csv'):
    acs_code_df = pd.read_csv(filepath)
    acs_dict = {}
    for i in range(len(acs_code_df)):
        acs_dict[acs_code_df.loc[i,'Variable Name']] = acs_code_df.loc[i,'Variable Code']

    code_list = ['NAME']
    for f in var_select:
        code_list.append(acs_dict[f])
    return tuple(code_list), acs_dict


In [36]:
code_list,acs_dict = get_acs_code(var_select=['Total Population', 'Total Housing Units'],filepath = '../data/acs_variable_code.csv')
print(code_list)

('NAME', 'B01003_001E', 'B25001_001E')


In [12]:
import requests
def get_fips_code(lat, lon):
    url = "https://geo.fcc.gov/api/census/block/find"
    params = {
        "latitude": lat,
        "longitude": lon,
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    state_fips = data["State"]["FIPS"]
    state_name = data["State"]["name"]
    county_fips = data["County"]["FIPS"]
    county_name = data["County"]["name"]
    return state_fips,state_name, county_fips, county_name


In [23]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


In [25]:
api_file = '/Users/caoguangyue/GitHub/Urban-Design-Equity-Metrics/app/census_api_key.txt'
api_key = get_file_contents(api_file)
c = Census(api_key)
# Replace with Input from lat, lng
state_fips, state_name, county_fips, county_name = get_fips_code(37.7749, -122.4194)
print(code_list)
print(state_fips, county_fips)


('NAME', 'B01003_001E', 'B25001_001E')
06 06075


In [29]:

sc_census = c.acs5.state_county_blockgroup(fields= code_list, 
                                           state_fips = str(state_fips), 
                                           county_fips = str(county_fips[2::]), 
                                           blockgroup = '*', 
                                           year = 2019)


In [37]:
#### 2. Feature engineering
# Feature Engineering
acs_df = pd.DataFrame(sc_census)
acs_df['geoid'] = sc_df['state'] + sc_df['county'] + sc_df['tract'] + sc_df['block group']


inv_acs_dict = {v: k for k, v in acs_dict.items()}
acs_df.rename(columns=inv_acs_dict, inplace=True)

acs_df.head()

,NAME,Total Population,Total Housing Units,state,county,tract,block group,geoid
0,"Block Group 4, Census Tract 452, San Francisco...",1029.0,469.0,06,075,045200,4,060750452004
1,"Block Group 2, Census Tract 452, San Francisco...",1838.0,892.0,06,075,045200,2,060750452002
2,"Block Group 5, Census Tract 452, San Francisco...",1493.0,539.0,06,075,045200,5,060750452005
3,"Block Group 1, Census Tract 111, San Francisco...",1755.0,1296.0,06,075,011100,1,060750111001
4,"Block Group 1, Census Tract 117, San Francisco...",859.0,586.0,06,075,011700,1,060750117001
